In [1]:
import os

print(os.getcwd())
print(os.listdir())

/Users/larscarlschmid/Documents/_repos/ChessOps
['statistics', 'add_bos.py', 'streamlit', 'products', 'requirements.txt', 'finetune.ipynb', 'env.yaml', 'output', 'checkDuplicates.py', 'models', 'README.md', 'pgn_to_xlan.py', 'logs', '.gitignore', 'dataPreProcessing.ipynb', 'remove_lines.py', 'pipeline.ipynb', 'create_tokens.py', 'pipeline.png', 'pipeline.yaml', 'venv', '.git', 'backup', 'data', 'inference.ipynb', 'finetune.py', 'src']


In [2]:
token_path = "./src/tokenizer/xlanplus_tokens.json"  # tokenizer needs this to know which tokens to use

In [3]:
from transformers import AutoModelForCausalLM, GPT2LMHeadModel

model_id = "Leon-LLM/Leon-Chess-350k-Plus"
# peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_carlsen_5E_0.0001LR"
# peft_model_id = "larscarl/Leon-Chess-350k-Plus_LoRA_kasparov_5E_0.0001LR"
model = AutoModelForCausalLM.from_pretrained(model_id)
# model.load_adapter(peft_model_id)

In [4]:
input_string = ""
number_of_games = 1000
plies = 125  # number of plies (half-moves) to predict => 125 seems to be the maximum
number_of_tokens_per_ply = 4
temperature = 0.2  # higher is more random, lower is more deterministic
max_batch_size = 16  # maximum number of predictions to make in parallel (higher is faster but requires more memory)

In [5]:
from src.generate_prediction import generate_batch_predictions

output_batch, predicted_tokens_batch, input_tokens_batch = generate_batch_predictions(
    inputs=[input_string]*number_of_games,
    num_tokens_to_generate=plies * number_of_tokens_per_ply,
    model=model,
    token_path=token_path,
    temperature=temperature,
    max_batch_size=max_batch_size,
)

print("output_batch: ", output_batch)
for game in output_batch:
    print(f"len(game.split(' ')): {len(game.split(' '))}")

output_batch:  ['Pe2e4- Pe7e6- Pd2d3- Pc7c5- Pf2f3- Nb8c6- Pc2c3- Pa7a6- Pg2g3- Pb7b5- Ph2h3- Bc8b7- Pb2b3- Pd7d5- Pa2a3- Pd5e4x Pf3e4x Ng8f6- Bf1g2- Bf8e7- Ng1f3- Ke8g8- Ke1g1- Qd8c7- Bc1f4- Qc7d7- Nb1d2- Ra8d8- Qd1e2- Qd7d3x Qe2d3x Rd8d3x Nf3e5- Nc6e5x Bf4e5x Nf6d7- Be5f4- Be7f6- Ra1d1- Rf8d8- Nd2f3- Nd7b6- Pe4e5- Bf6e7- Nf3g5- Be7g5x Bf4g5x Rd8d7- Bg2b7x Rd7b7x Bg5f4- Nb6d5- Bf4d2- Pc5c4- Pb3b4- Pa6a5- Pb4a5x Rb7a7- Rd1a1- Ra7a5x Ra1b1- Ra5a3x Rb1b5x Ra3c3x Bd2c3x Nd5c3x Rb5b8+ Kg8h7- Rf1f7x Nc3e2+ Kg1f2- Ne2d4- Rf7g7$ Kh7h6- Rg7h7+ Kh6g6- Rh7g7+ Kg6f5- Rg7f7+ Kf5e5x Rb8e8- Pc4c3- Re8c8- Rc3c8x Rf7e7- Rc8c2+ Kf2e3- Nd4f5+ Ke3d3x Nf5e7x Kd3c2x Ne7d5- Kc2d3- Ke5f5- Kd3d4- Kf5g5- Kd4e4- Pe6e5- Ke4f3- Pe5e4+ Kf3e4x Nd5f6+ Ke4f3- Nf6h5- Pg3g4- Nh5f6- Kf3g3- Nf6e4+ Kg3g2- Ne4f6- Kg2g3- Nf6e4+ Kg3g2- Ne4d2- Kg2g3- Nd2e4+ Kg3g2- Ne4f6- Kg2g3- Nf6e4+ Kg3g2- Ne4f6- Kg2g3- Nf6e4# 0-1\n', 'Pe2e4- Pe7e5- Pd2d3- Pd7d6- Pf2f4- Nb8c6- Pf4f5- Pf7f6- Ng1f3- Bc8d7- Bf1e2- Ng8e7- Ke1g1- Ne7c8- Pc2c3- B

In [6]:
from src.validation.validate_sequence import evaluate_sequence

result = evaluate_sequence(output_batch, predicted_tokens_batch, debug=False, notation="xLANplus")

# averages of 'number of moves until error'
sum_moves = 0
for game in result:
    sum_moves += game[1]
average_moves = sum_moves / len(result)
print(f"Average number of moves until error: {average_moves}")

Average number of moves until error: 58.985


In [ ]:
model_name = peft_model_id.split("/")[-1]
model_name

In [11]:
# write data to file
# csv should have the following columns:
# 'Game String' (full game generated by model) = result[i][0]
# 'Moves Until Error' = result[i][1]
# 'Error Type' = result[i][2]
# 'First Illegal Move' (game string up to the first illegal move) = result[i][3]
# 'Predicted Tokens' (full game in tokenized form) = predicted_tokens_batch[i]

import pandas as pd

# output_path = f"./statistics/statistical_evaluations{model_name}_sequences_{number_of_games}x{plies}.csv"
output_path = f"./statistics/statistical_evaluations_{model_id.split('/')[-1]}_sequences_{number_of_games}x{plies}.csv"

data = {
    'Game String': [game[0] for game in result],
    'Moves Until Error': [game[1] for game in result],
    'Error Type': [game[2] for game in result],
    'First Illegal Move': [game[3] for game in result],
    'Predicted Tokens': predicted_tokens_batch
}

# for every game in 'First Illegal Move' (game[3]), convert the sequence to PGN

df = pd.DataFrame(data)
df.to_csv(output_path, index=False)

print(f"Data written to {output_path}")

Data written to ./statistics/statistical_evaluations_Leon-Chess-350k-Plus_sequences_1000x125.csv
